<a href="https://colab.research.google.com/github/HarithBolhi/ECE633/blob/main/Week_10A_yolinux_POSIX_thread_(pthread)_libraries_(using_C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 10A
Ir. Dr. Syahrul Afzal bin Che Abdullah
## yolinux POSIX thread (pthread) libraries (using C)
[Linux Tutorial: POSIX Threads](http://www.yolinux.com/TUTORIALS/LinuxTutorialPosixThreads.html)

### Apparatus
1. Welcome To Colaboratory - Colaboratory <br>
https://colab.research.google.com/
2. Online C Compiler, Visual Debugger, and AI Tutor - Learn C programming by visualizing code <br>
https://pythontutor.com/c.html#mode=edit
3. GDB online Debugger | Compiler - Code, Compile, Run, Debug online C, C++ <br>
https://www.onlinegdb.com/ <br>
https://www.onlinegdb.com/online_c_compiler


### Sourcecode
<font color='red'>Note: There are several ways to upload a file to Google Colab, like mounting your Google Drive and execute a copy operation.</font>

### Thread Basics:



### Thread Creation and Termination:
Function call: pthread_create - create a new thread. <br>
Function call: pthread_join - wait for termination of another thread. <br>
Function call: pthread_exit - terminate the calling thread.

In [ ]:
%%writefile pthread1.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

void *print_message_function( void *ptr );

//main()
int main() // modify by sabca
{
     pthread_t thread1, thread2;
     const char *message1 = "Thread 1";
     const char *message2 = "Thread 2";
     int  iret1, iret2;

    /* Create independent threads each of which will execute function */

     iret1 = pthread_create( &thread1, NULL, print_message_function, (void*) message1);
     if(iret1)
     {
         fprintf(stderr,"Error - pthread_create() return code: %d\n",iret1);
         exit(EXIT_FAILURE);
     }

     iret2 = pthread_create( &thread2, NULL, print_message_function, (void*) message2);
     if(iret2)
     {
         fprintf(stderr,"Error - pthread_create() return code: %d\n",iret2);
         exit(EXIT_FAILURE);
     }

     printf("pthread_create() for thread 1 returns: %d\n",iret1);
     printf("pthread_create() for thread 2 returns: %d\n",iret2);

     /* Wait till threads are complete before main continues. Unless we  */
     /* wait we run the risk of executing an exit which will terminate   */
     /* the process and all threads before the threads have completed.   */

     pthread_join( thread1, NULL);
     pthread_join( thread2, NULL);

     exit(EXIT_SUCCESS);
}

void *print_message_function( void *ptr )
{
     char *message;
     message = (char *) ptr;
     printf("%s \n", message);

     return NULL; //add by sabca
}

Writing pthread1.c


In [ ]:
!gcc -o pthread1 pthread1.c -lpthread -Wall
!./pthread1

pthread_create() for thread 1 returns: 0
pthread_create() for thread 2 returns: 0
Thread 2 
Thread 1 


### Thread Synchronization:
The threads library provides three synchronization mechanisms: <br>
1. mutexes - Mutual exclusion lock: Block access to variables by other threads. This enforces exclusive access by a thread to a variable or set of variables. <br><br>
pthread_mutex_lock() - acquire a lock on the specified mutex variable. If the mutex is already locked by another thread, this call will block the calling thread until the mutex is unlocked. <br>
pthread_mutex_unlock() - unlock a mutex variable. An error is returned if mutex is already unlocked or owned by another thread. <br>
pthread_mutex_trylock() - attempt to lock a mutex or will return error code if busy. Useful for preventing deadlock conditions.

In [ ]:
%%writefile mutex1.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

void *functionC();
pthread_mutex_t mutex1 = PTHREAD_MUTEX_INITIALIZER;
int  counter = 0;

//main()
int main() // modify by sabca
{
   int rc1, rc2;
   pthread_t thread1, thread2;

   /* Create independent threads each of which will execute functionC */

   if( (rc1=pthread_create( &thread1, NULL, &functionC, NULL)) )
   {
      printf("Thread creation failed: %d\n", rc1);
   }

   if( (rc2=pthread_create( &thread2, NULL, &functionC, NULL)) )
   {
      printf("Thread creation failed: %d\n", rc2);
   }

   /* Wait till threads are complete before main continues. Unless we  */
   /* wait we run the risk of executing an exit which will terminate   */
   /* the process and all threads before the threads have completed.   */

   pthread_join( thread1, NULL);
   pthread_join( thread2, NULL);

   exit(EXIT_SUCCESS);
}

void *functionC()
{
   pthread_mutex_lock( &mutex1 );
   counter++;
   printf("Counter value: %d\n",counter);
   pthread_mutex_unlock( &mutex1 );

   return NULL; //add by sabca
}

Writing mutex1.c


In [ ]:
!gcc -o mutex1 mutex1.c -lpthread -Wall
!./mutex1

Counter value: 1
Counter value: 2


In [ ]:
%%writefile mutex1_mod.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

void *functionC();
pthread_mutex_t mutex1 = PTHREAD_MUTEX_INITIALIZER;
int  counter = 0;

//main()
int main() // modify by sabca
{
   int rc1, rc2;
   pthread_t thread1, thread2;

   /* Create independent threads each of which will execute functionC */

   if( (rc1=pthread_create( &thread1, NULL, &functionC, NULL)) )
   {
      printf("Thread creation failed: %d\n", rc1);
   }

   if( (rc2=pthread_create( &thread2, NULL, &functionC, NULL)) )
   {
      printf("Thread creation failed: %d\n", rc2);
   }

   /* Wait till threads are complete before main continues. Unless we  */
   /* wait we run the risk of executing an exit which will terminate   */
   /* the process and all threads before the threads have completed.   */

   pthread_join( thread1, NULL);
   pthread_join( thread2, NULL);

   exit(EXIT_SUCCESS);
}

void *functionC()
{
   pthread_mutex_lock( &mutex1 );
   for (int i = 0; i < 10000000; i++) {
      counter++;
   }
   printf("Counter value: %d\n",counter);
   pthread_mutex_unlock( &mutex1 );

   return NULL; //add by sabca
}

Overwriting mutex1_mod.c


In [ ]:
!gcc -o mutex1_mod mutex1_mod.c -lpthread -Wall
!./mutex1_mod

Counter value: 10000000
Counter value: 20000000


2. joins - Make a thread wait till others are complete (terminated). <br><br>
pthread_self() - return identifier of current thread.

In [ ]:
%%writefile join1.c
#include <stdio.h>
#include <pthread.h>

#define NTHREADS 10
void *thread_function(void *);
pthread_mutex_t mutex1 = PTHREAD_MUTEX_INITIALIZER;
int  counter = 0;

//main()
int main() // modify by sabca
{
   pthread_t thread_id[NTHREADS];
   int i, j;

   for(i=0; i < NTHREADS; i++)
   {
      pthread_create( &thread_id[i], NULL, thread_function, NULL );
   }

   for(j=0; j < NTHREADS; j++)
   {
      pthread_join( thread_id[j], NULL);
   }

   /* Now that all threads are complete I can print the final result.     */
   /* Without the join I could be printing a value before all the threads */
   /* have been completed.                                                */

   printf("Final counter value: %d\n", counter);
}

void *thread_function(void *dummyPtr)
{
   printf("Thread number %ld\n", pthread_self());
   pthread_mutex_lock( &mutex1 );
   counter++;
   pthread_mutex_unlock( &mutex1 );

   return NULL; //add by sabca
}

Writing join1.c


In [ ]:
!gcc -o join1 join1.c -lpthread -Wall
!./join1

Thread number 138843413829184
Thread number 138843472578112
Thread number 138843464185408
Thread number 138843455792704
Thread number 138843480970816
Thread number 138843447400000
Thread number 138843489363520
Thread number 138843439007296
Thread number 138843430614592
Thread number 138843422221888
Final counter value: 10


3. condition variables - data type pthread_cond_t. <br>

Functions used in conjunction with the condition variable:<br>
**Creating/Destroying:** <br>
*   pthread_cond_init <br>
*   pthread_cond_t cond = PTHREAD_COND_INITIALIZER; <br>
*   pthread_cond_destroy <br>

**Waiting on condition:** <br>
*   pthread_cond_wait - unlocks the mutex and waits for the condition variable cond to be signaled. <br>
*   pthread_cond_timedwait - place limit on how long it will block. <br>

**Waking thread based on condition:** <br>
*   pthread_cond_signal - restarts one of the threads that are waiting on the condition variable cond. <br>
*   pthread_cond_broadcast - wake up all threads blocked by the specified condition variable. <br>

In [ ]:
%%writefile cond1.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

pthread_mutex_t count_mutex     = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t  condition_var   = PTHREAD_COND_INITIALIZER;

void *functionCount1();
void *functionCount2();
int  count = 0;
#define COUNT_DONE  10
#define COUNT_HALT1  3
#define COUNT_HALT2  6

//main()
int main() // modify by sabca
{
   pthread_t thread1, thread2;

   pthread_create( &thread1, NULL, &functionCount1, NULL);
   pthread_create( &thread2, NULL, &functionCount2, NULL);

   pthread_join( thread1, NULL);
   pthread_join( thread2, NULL);

   printf("Final count: %d\n",count);

   exit(EXIT_SUCCESS);
}

// Write numbers 1-3 and 8-10 as permitted by functionCount2()

void *functionCount1()
{
   for(;;)
   {
      // Lock mutex and then wait for signal to relase mutex
      pthread_mutex_lock( &count_mutex );

      // Wait while functionCount2() operates on count
      // mutex unlocked if condition varialbe in functionCount2() signaled.
      pthread_cond_wait( &condition_var, &count_mutex );
      count++;
      printf("Counter value functionCount1: %d\n",count);

      pthread_mutex_unlock( &count_mutex );

      if(count >= COUNT_DONE) return(NULL);
    }
}

// Write numbers 4-7

void *functionCount2()
{
    for(;;)
    {
       pthread_mutex_lock( &count_mutex );

       if( count < COUNT_HALT1 || count > COUNT_HALT2 )
       {
          // Condition of if statement has been met.
          // Signal to free waiting thread by freeing the mutex.
          // Note: functionCount1() is now permitted to modify "count".
          pthread_cond_signal( &condition_var );
       }
       else
       {
          count++;
          printf("Counter value functionCount2: %d\n",count);
       }

       pthread_mutex_unlock( &count_mutex );

       if(count >= COUNT_DONE) return(NULL);
    }
}

Writing cond1.c


In [ ]:
!gcc -o cond1 cond1.c -lpthread -Wall
!./cond1

Counter value functionCount1: 1
Counter value functionCount1: 2
Counter value functionCount1: 3
Counter value functionCount2: 4
Counter value functionCount2: 5
Counter value functionCount2: 6
Counter value functionCount2: 7
Counter value functionCount1: 8
Counter value functionCount1: 9
Counter value functionCount1: 10
Final count: 10


In [ ]:
%%writefile cond1_mod.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

pthread_mutex_t count_mutex     = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t  condition_var   = PTHREAD_COND_INITIALIZER;

void *functionCount1();
void *functionCount2();
int  count = 0;
#define COUNT_DONE  10
#define COUNT_HALT1  3
#define COUNT_HALT2  6

//main()
int main() // modify by sabca
{
   pthread_t thread1, thread2;

   pthread_create( &thread1, NULL, &functionCount1, NULL);
   pthread_create( &thread2, NULL, &functionCount2, NULL);

   pthread_join( thread1, NULL);
   pthread_join( thread2, NULL);

   printf("Final count: %d\n",count);

   exit(EXIT_SUCCESS);
}

// Write numbers 1-3 and 8-10 as permitted by functionCount2()

void *functionCount1()
{
   for(;;)
   {
      // Lock mutex and then wait for signal to relase mutex
      pthread_mutex_lock( &count_mutex );

      // Wait while functionCount2() operates on count
      // mutex unlocked if condition varialbe in functionCount2() signaled.
      pthread_cond_wait( &condition_var, &count_mutex );
      count++;
      printf("Thread number %ld ", pthread_self());
      printf("Counter value functionCount1: %d\n",count);

      pthread_mutex_unlock( &count_mutex );

      if(count >= COUNT_DONE) return(NULL);
    }
}

// Write numbers 4-7

void *functionCount2()
{
    for(;;)
    {
       pthread_mutex_lock( &count_mutex );

       if( count < COUNT_HALT1 || count > COUNT_HALT2 )
       {
          // Condition of if statement has been met.
          // Signal to free waiting thread by freeing the mutex.
          // Note: functionCount1() is now permitted to modify "count".
          pthread_cond_signal( &condition_var );
       }
       else
       {
          count++;
          printf("Thread number %ld ", pthread_self());
          printf("Counter value functionCount2: %d\n",count);
       }

       pthread_mutex_unlock( &count_mutex );

       if(count >= COUNT_DONE) return(NULL);
    }
}

Writing cond1_mod.c


In [ ]:
!gcc -o cond1_mod cond1_mod.c -lpthread -Wall
!./cond1_mod

Thread number 135606961653312 Counter value functionCount1: 1
Thread number 135606961653312 Counter value functionCount1: 2
Thread number 135606961653312 Counter value functionCount1: 3
Thread number 135606953260608 Counter value functionCount2: 4
Thread number 135606953260608 Counter value functionCount2: 5
Thread number 135606953260608 Counter value functionCount2: 6
Thread number 135606953260608 Counter value functionCount2: 7
Thread number 135606961653312 Counter value functionCount1: 8
Thread number 135606961653312 Counter value functionCount1: 9
Thread number 135606961653312 Counter value functionCount1: 10
Final count: 10


### Thread Scheduling:

### Thread Pitfalls:
Race conditions: While the code may appear on the screen in the order you wish the code to execute, threads are scheduled by the operating system and are executed at random. It cannot be assumed that threads are executed in the order they are created. They may also execute at different speeds. When threads are executing (racing to complete) they may give unexpected results (race condition). Mutexes and joins must be utilized to achieve a predictable execution order and outcome. <br>
Thread safe code: The threaded routines must call functions which are "thread safe". This means that there are no static or global variables which other threads may clobber or read assuming single threaded operation. If static or global variables are used then mutexes must be applied or the functions must be re-written to avoid the use of these variables. In C, local variables are dynamically allocated on the stack. Therefore, any function that does not use static data or other shared resources is thread-safe. Thread-unsafe functions may be used by only one thread at a time in a program and the uniqueness of the thread must be ensured. Many non-reentrant functions return a pointer to static data. This can be avoided by returning dynamically allocated data or using caller-provided storage. An example of a non-thread safe function is strtok which is also not re-entrant. The "thread safe" version is the re-entrant version strtok_r. <br>
Mutex Deadlock: This condition occurs when a mutex is applied but then not "unlocked". This causes program execution to halt indefinitely. It can also be caused by poor application of mutexes or joins. Be careful when applying two or more mutexes to a section of code. If the first pthread_mutex_lock is applied and the second pthread_mutex_lock fails due to another thread applying a mutex, the first mutex may eventually lock all other threads from accessing data including the thread which holds the second mutex. The threads may wait indefinitely for the resource to become free causing a deadlock. It is best to test and if failure occurs, free the resources and stall before retrying.

In [ ]:
%%writefile test1.c
...
pthread_mutex_lock(&mutex_1);
while ( pthread_mutex_trylock(&mutex_2) )  /* Test if already locked   */
{
   pthread_mutex_unlock(&mutex_1);  /* Free resource to avoid deadlock */
   ...
   /* stall here   */
   ...
   pthread_mutex_lock(&mutex_1);
}
count++;
pthread_mutex_unlock(&mutex_1);
pthread_mutex_unlock(&mutex_2);
...

The order of applying the mutex is also important. The following code segment illustrates a potential for deadlock:

In [ ]:
%%writefile test2.c
void *function1()
{
   ...
   pthread_mutex_lock(&lock1);           // Execution step 1
   pthread_mutex_lock(&lock2);           // Execution step 3 DEADLOCK!!!
   ...
   ...
   pthread_mutex_lock(&lock2);
   pthread_mutex_lock(&lock1);
   ...
}

void *function2()
{
   ...
   pthread_mutex_lock(&lock2);           // Execution step 2
   pthread_mutex_lock(&lock1);
   ...
   ...
   pthread_mutex_lock(&lock1);
   pthread_mutex_lock(&lock2);
   ...
}

main()
{
   ...
   pthread_create(&thread1, NULL, function1, NULL);
   pthread_create(&thread2, NULL, function2, NULL);
   ...
}

In [ ]:
%%writefile mutex1_mod.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h> // header file for sleep()

void *function1();
void *function2();
pthread_mutex_t lock1 = PTHREAD_MUTEX_INITIALIZER;
pthread_mutex_t lock2 = PTHREAD_MUTEX_INITIALIZER;
int  counter = 0;

//main()
int main() // modify by sabca
{
   int rc1, rc2;
   pthread_t thread1, thread2;

   /* Create independent threads each of which will execute functionC */

   if( (rc1=pthread_create( &thread1, NULL, &function1, NULL)) )
   {
      printf("Thread creation failed: %d\n", rc1);
   }

   if( (rc2=pthread_create( &thread2, NULL, &function2, NULL)) )
   {
      printf("Thread creation failed: %d\n", rc2);
   }

   /* Wait till threads are complete before main continues. Unless we  */
   /* wait we run the risk of executing an exit which will terminate   */
   /* the process and all threads before the threads have completed.   */

   pthread_join( thread1, NULL);
   pthread_join( thread2, NULL);

   exit(EXIT_SUCCESS);
}

void *function1()
{
   printf("In function 1.");
   pthread_mutex_lock( &lock1 );
   printf("Thread 1 got first lock.");
   sleep(1);
   pthread_mutex_lock( &lock2 );
   printf("Thread 1 got second lock.");
   sleep(1);
   counter++;
   printf("Counter value function 1: %d\n",counter);
   pthread_mutex_unlock( &lock2 );
   pthread_mutex_unlock( &lock1 );

   return NULL; //add by sabca
}

void *function2()
{
   printf("In function 2.");
   pthread_mutex_lock( &lock1 );
   printf("Thread 2 got first lock.");
   sleep(1);
   pthread_mutex_lock( &lock2 );
   printf("Thread 2 got second lock.");
   sleep(1);
   counter++;
   printf("Counter value function 2: %d\n",counter);
   pthread_mutex_unlock( &lock2 );
   pthread_mutex_unlock( &lock1 );

   return NULL; //add by sabca
}

Overwriting mutex1_mod.c


In [ ]:
!gcc -o mutex1_mod mutex1_mod.c -lpthread -Wall
!./mutex1_mod

In function 2.Thread 2 got first lock.In function 1.Thread 2 got second lock.Counter value function 2: 1
Thread 1 got first lock.Thread 1 got second lock.Counter value function 1: 2


In [ ]:
%%writefile mutex1_mod_deadlock.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h> // header file for sleep()

void *function1();
void *function2();
pthread_mutex_t lock1 = PTHREAD_MUTEX_INITIALIZER;
pthread_mutex_t lock2 = PTHREAD_MUTEX_INITIALIZER;
int  counter = 0;

//main()
int main() // modify by sabca
{
   int rc1, rc2;
   pthread_t thread1, thread2;

   /* Create independent threads each of which will execute functionC */

   if( (rc1=pthread_create( &thread1, NULL, &function1, NULL)) )
   {
      printf("Thread creation failed: %d\n", rc1);
   }

   if( (rc2=pthread_create( &thread2, NULL, &function2, NULL)) )
   {
      printf("Thread creation failed: %d\n", rc2);
   }

   /* Wait till threads are complete before main continues. Unless we  */
   /* wait we run the risk of executing an exit which will terminate   */
   /* the process and all threads before the threads have completed.   */

   pthread_join( thread1, NULL);
   pthread_join( thread2, NULL);

   exit(EXIT_SUCCESS);
}

void *function1()
{
   printf("In function 1.");
   pthread_mutex_lock( &lock1 );
   printf("Thread 1 got first lock.");
   sleep(1);
   pthread_mutex_lock( &lock2 );
   printf("Thread 1 got second lock.");
   sleep(1);
   counter++;
   printf("Counter value function 1: %d\n",counter);
   pthread_mutex_unlock( &lock2 );
   pthread_mutex_unlock( &lock1 );

   return NULL; //add by sabca
}

void *function2()
{
   printf("In function 2.");
   pthread_mutex_lock( &lock2 ); // change the ordering
   printf("Thread 2 got first lock.");
   sleep(1);
   pthread_mutex_lock( &lock1 ); // change the ordering
   printf("Thread 2 got second lock.");
   sleep(1);
   counter++;
   printf("Counter value function 2: %d\n",counter);
   pthread_mutex_unlock( &lock1 );
   pthread_mutex_unlock( &lock2 );

   return NULL; //add by sabca
}

Writing mutex1_mod_deadlock.c


In [ ]:
!gcc -o mutex1_mod_deadlock mutex1_mod_deadlock.c -lpthread -Wall
!./mutex1_mod_deadlock

^C


In [ ]:
!diff -u mutex1_mod.c mutex1_mod_deadlock.c

--- mutex1_mod.c	2024-06-04 07:30:42.591364580 +0000
+++ mutex1_mod_deadlock.c	2024-06-04 07:31:09.168269462 +0000
@@ -57,16 +57,16 @@
 void *function2()
 {
    printf("In function 2.");
-   pthread_mutex_lock( &lock1 );
+   pthread_mutex_lock( &lock2 ); // change the ordering
    printf("Thread 2 got first lock.");
    sleep(1);
-   pthread_mutex_lock( &lock2 );
+   pthread_mutex_lock( &lock1 ); // change the ordering
    printf("Thread 2 got second lock.");
    sleep(1);
    counter++;
    printf("Counter value function 2: %d\n",counter);
-   pthread_mutex_unlock( &lock2 );
    pthread_mutex_unlock( &lock1 );
+   pthread_mutex_unlock( &lock2 );
 
    return NULL; //add by sabca
 }


### Thread Debugging: